In [16]:
import pandas as pd
import numpy as np
import csv
import gzip
import re

In [17]:
import operator

In [18]:
import sys
if (sys.version_info > (3, 0)):
    import pickle as pkl
else: #Python 2.7 imports
    import cPickle as pkl

In [19]:
from keras.models import load_model
from keras.preprocessing import sequence

C:\Users\Aravindhan.Poopathy\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
wholeInputDf = pd.read_csv('C:/Users/Aravindhan.Poopathy/OneDrive - So Energy/Arav/Tickets/finalemail.csv')

In [21]:
testDf = wholeInputDf.head(5)
testDf

,Unnamed: 0,user-id,source-name,Subject,description,priority,priority-name,st-survey-rating,sentiment
0,0,7.002914e+09,Email,You lost my cat!,"Dear Customer Relations,\r\r\n\r\r\n\r\r\nI re...",3.0,High,103.0,0
1,1,NaN,Email,Need help with payments,Credit card failed.\r\r\n\r\r\n,3.0,High,102.0,1
2,2,NaN,Email,Re: Acc. No: 00159568 - Your Electricity Readings,Hi Shahid\r\r\n\r\r\n\r\r\nPlease would you ve...,1.0,Low,103.0,0
3,3,7.003836e+09,Outbound Email,So Energy - Two Suppliers Billing Your Electri...,"Good Afternoon Vicencio, \r\r\n \r\r\n Thank...",1.0,Low,102.0,1
4,4,NaN,Email,Lost email,Hi\r\r\n\r\r\nYesterday Sunday 12 November you...,1.0,Low,103.0,0


In [22]:
word2Idx_path = open("C:/Users/Aravindhan.Poopathy/OneDrive - So Energy/Arav/Sentence_classic_new-master/Sentence_classic_new-master/code/soeMailData/pkl/word2Idx.pickle", "rb")
word2Idx = pkl.load(word2Idx_path)

In [23]:
loadedModel = load_model("C:/Users/Aravindhan.Poopathy/OneDrive - So Energy/Arav/Sentence_classic_new-master/Sentence_classic_new-master/code/soeMailData/soemailModels/classMailModel1.h5")

In [24]:
def createtestMatrix(sentence, word2Idx):
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    


    testMatrix = []
    unknownWordCount = 0
    wordCount = 0

    targetWordIdx = 0

    for word in sentence:
        wordCount += 1

        if word in word2Idx:
            wordIdx = word2Idx[word]
        elif word.lower() in word2Idx:
            wordIdx = word2Idx[word.lower()]
        else:
            wordIdx = unknownIdx
            unknownWordCount += 1
        testMatrix.append(wordIdx)
#     print("Unknown tokens in test_Text: %.2f%%" % (unknownWordCount/(float(wordCount))*100))

    return testMatrix

data = pkl.load(gzip.open("C:/Users/Aravindhan.Poopathy/OneDrive - So Energy/Arav/Sentence_classic_new-master/Sentence_classic_new-master/code/soeMailData/pkl/maildata.pkl.gz","rb"))
print("data loaded!")

train_labels = data['train']['labels']
train_sentences = data['train']['sentences']

dev_labels = data['dev']['labels']
dev_sentences = data['dev']['sentences']

test_labels = data['test']['labels']
test_sentences = data['test']['sentences']

word_embeddings = data['wordEmbeddings']

# :: Find the longest sentence in our dataset ::
max_sentence_len = 0
for sentence in train_sentences + dev_sentences + test_sentences:
    max_sentence_len = max(len(sentence), max_sentence_len)

print("Longest sentence: %d" % max_sentence_len)

batch_size = 50


data loaded!
Longest sentence: 704


In [25]:
def sentiment(row):
    mail = row['description']
    mail = re.sub(r'[?|$|>|!|<|#|*|[|-|]|_|�|//|\\', '', mail)
    mail = mail.replace('\ufeff','')
    mail = re.sub(r'[\w\.-]+@[\w\.-]+','',mail)
    mail = re.sub(r"http\S+",'',mail)
    mail = re.sub(r"[\d{1,2}\/\d{1,2}\/\d{4}]+",'',mail)
    mail = " ".join(filter(lambda x:x[0] not in ['@','!','*','&','/','\\','#','-','='], mail.split()))
    mail = re.sub(r'\ P.S. .+ smart meters','',mail)
    mail = re.sub(r'\ P.S. This is a sample email .+ helpdesk','',mail)
    mail = re.sub(r'\ Get in touch: .+ All rights reserved.','',mail)
    mail = re.sub(r'\ This message contains confidential information.+may not be attributed to So Energy.', '', mail)
    mail = re.sub(r'\ This message contains confidential information.+a hard-copy version', '', mail)
    mail = re.sub(r'\ This message contains confidential information.+','',mail)
    mail = re.sub(r'\ message contains confidential.+','', mail)
    mail = re.sub(r'\ So Energy Trading Limited.+','',mail)
    mail = re.sub(r'\ Like us on.+','',mail)
    mail = re.sub(r'\ Thanks The So Energy TeamLike.+','',mail)
    mail = re.sub(r'\sent from my.+','', mail)
    mail = re.sub(r'\ So Energy wrote:.+text-decoration:underline;yiv','',mail)
    mail = re.sub(r'\ So Energy wrote:.+All rights reserved.','',mail)
    mail = re.sub(r'\ So wrote:.+All rights reserved.','',mail)
    mail = re.sub(r'\ So Energyyiv.+acolor:fcd;font-weight:normal;text-decoration:underline;yiv','',mail)
    mail = re.sub(r'\So Energy wrote.+text-decoration:underline; yiv ','',mail)
    mail = re.sub(r"\bwww.+","",mail)
    mail = re.sub(r"\So Energy p margin:px.+text-decoration:underline;","",mail)
    mail = re.sub(r"\(.+\)","",mail)
    mail = re.sub("\{.+\}","",mail)
    mail = re.sub(r"\.yivReadMsgBody.+collapse;yiv","",mail)
    mail = re.sub(r"\Sent from.+A;padding:px;border-radius:px;background-color:A;yiv","",mail)
    mail = re.sub(r"\Sent from.+pmargin-top:px;margin-bottom:px;","",mail)
    mail = re.sub(r"\Sent from.+To: Subject:","",mail)    
    
    
    mailnew = mail.split()
    
    finalTest_matrix = createtestMatrix(mailnew, word2Idx)

    testarray = np.array(finalTest_matrix)
    testarray = testarray.reshape(1,len(finalTest_matrix))

    testarray.tolist()
    
    result_X = sequence.pad_sequences(testarray, maxlen=max_sentence_len)

    result_y_pred = loadedModel.predict(result_X, batch_size=batch_size)
    result_y_pred = result_y_pred.tolist()
    result = result_y_pred[0]

#     print ('Prediction result as a whole : ',result)
    
    
    index, value = max(enumerate(result), key=operator.itemgetter(1))
    
    if(index==0):
#         print("The User Rating is Excellent! ", topicList)
        senti = 'Excellent'
    elif(index==1):
#         print("The user Rating is Good! ", topicList)
        senti = 'Good'
    elif(index==2):
#         print("The user Rating is Average! ", topicList)
        senti = 'Average'
    elif(index==3):
#         print("The user Rating is Bad! ", topicList)
        senti = 'Bad'
    elif(index==4):
#         print("The user Rating is Terrible ", topicList)
        senti = 'Terrible'
    
    return(senti)


In [26]:
# testDf.apply(sentiment, axis=1)
desc = wholeInputDf.apply(sentiment, axis=1)
print(desc)

0       Excellent
1            Good
2       Excellent
3            Good
4            Good
5            Good
6            Good
7            Good
8            Good
9            Good
10           Good
11      Excellent
12      Excellent
13      Excellent
14           Good
15           Good
16      Excellent
17      Excellent
18      Excellent
19           Good
20           Good
21           Good
22           Good
23           Good
24      Excellent
25           Good
26           Good
27           Good
28      Excellent
29           Good
          ...    
4325         Good
4326    Excellent
4327         Good
4328         Good
4329         Good
4330         Good
4331         Good
4332    Excellent
4333         Good
4334         Good
4335         Good
4336    Excellent
4337      Average
4338         Good
4339         Good
4340         Good
4341         Good
4342         Good
4343         Good
4344         Good
4345         Good
4346         Good
4347         Good
4348         Good
4349      

In [27]:
wholeInputDf['Pred_sentiment'] = wholeInputDf.apply(sentiment, axis=1)

In [28]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):

    display(wholeInputDf) 

,Unnamed: 0,user-id,source-name,Subject,description,priority,priority-name,st-survey-rating,sentiment,Pred_sentiment
0,0,7.002914e+09,Email,You lost my cat!,"Dear Customer Relations,\r\r\n\r\r\n\r\r\nI re...",3.0,High,103.0,0,Excellent
1,1,NaN,Email,Need help with payments,Credit card failed.\r\r\n\r\r\n,3.0,High,102.0,1,Good
2,2,NaN,Email,Re: Acc. No: 00159568 - Your Electricity Readings,Hi Shahid\r\r\n\r\r\n\r\r\nPlease would you ve...,1.0,Low,103.0,0,Excellent
3,3,7.003836e+09,Outbound Email,So Energy - Two Suppliers Billing Your Electri...,"Good Afternoon Vicencio, \r\r\n \r\r\n Thank...",1.0,Low,102.0,1,Good
4,4,NaN,Email,Lost email,Hi\r\r\n\r\r\nYesterday Sunday 12 November you...,1.0,Low,103.0,0,Good
5,5,NaN,Email,RE: Increasing dd amount,Thank you. Could i increase to £52 per month s...,1.0,Low,103.0,0,Good
6,6,NaN,Email,tariff change,Account No 00128947. I am presently on Gorill...,1.0,Low,103.0,0,Good
7,7,NaN,Email,RE: Your latest statement,I am unable to log on to my account and would ...,1.0,Low,102.0,1,Good
8,8,NaN,Email,Readings,Hello\r\r\n\r\r\nThese are the readings God wh...,1.0,Low,102.0,1,Good
9,9,NaN,Email,Re: We've adjusted your Direct Debit amount,Could you please leave at £61 per month ? We'r...,1.0,Low,103.0,0,Good


In [30]:
del wholeInputDf['Unnamed: 0']

In [31]:
wholeInputDf[wholeInputDf['Pred_sentiment'].isna()]

,user-id,source-name,Subject,description,priority,priority-name,st-survey-rating,sentiment,Pred_sentiment


In [32]:
wholeInputDf['st-survey-rating'].unique()

array([ 103.,  102., -102.,  100., -103.])

In [33]:
wholeInputDf['Pred_sentiment'].unique()

array(['Excellent', 'Good', 'Terrible', 'Average', 'Bad'], dtype=object)

In [35]:
wholeInputDf.to_csv('C:/Users/Aravindhan.Poopathy/OneDrive - So Energy/Arav/Sentence_classic_new-master/Sentence_classic_new-master/code/soeMailData/predresults/classmail_results.csv',encoding='utf-8')

In [6]:
wholeInputDf = pd.read_csv('C:/Users/Aravindhan.Poopathy/OneDrive - So Energy/Arav/Sentence_classic_new-master/Sentence_classic_new-master/code/soeMailData/predresults/classmail_results.csv',encoding='utf-8')